In [1]:
#importing the required packages
import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
def retrieving_information_for_the_day(year,month,day):
  # iterate the dates to get news from these dates
  web_url= "https://www.thehindu.com/archive/web/year/month/day/"
  #getting html source code from the url
  raw_data=requests.get(web_url,'lxml')
  #We create a Beautiful soup object and use it to find the required information from the url
  soup=bs(raw_data.content,'html5lib')
  information=soup.findAll('ul',class_='archive-list')
  return information

In [3]:
#the headlines from the day
def headlines(information):
  headlines=[]
  for i in range(len(information)):
      for j in range(len(information[i].findAll('a'))):
          headlines.append(information[i].findAll('a')[j])
  return headlines

In [4]:
#this is a function to scrap the title of the news and the link to the main content from the information
def links_and_headings(headlines):
    links=[]
    news_titles=[]
    for i in range(len(headlines)):
        news_titles.append(headlines[i].get_text())
        links.append(headlines[i]['href'])
    return links,news_titles

In [5]:
#this function scraps the authors of the articles and removes the articles with no authors.
def authors_and_reprocessing_data(links):
    authors=[]
    noauthors=[]
    c=0
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        author=soup1.find('a',{"class":["auth-nm lnk","auth-nm no-lnk"]})
        if author!=None:
            authors.append(author.get_text().strip('\n'))
        if author==None:
            noauthors.append(i)
            authors.append(0)
    return authors,noauthors

In [6]:
#this function goes to the links of the headines and gets the full content
def retrieving_news_content(links,headlines):
    news_contents=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        raw_content=soup1.findAll('p',class_="")
        paragraph=''
        for j in range(len(raw_content)):
            if len(raw_content[j].get_text())>10:
                paragraph+=raw_content[j].get_text()
        news_contents.append(paragraph.strip('\n').split("Start your 14 days trial now.")[0])
    return news_contents

In [7]:
def date_and_time(links):
    date=[]
    time=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        when=soup1.find('meta',{"name":"publish-date"})
        date_and_time=when['content'].split('T')
        date.append(date_and_time[0])
        time.append(date_and_time[1].replace("+05:30"," IST"))
    return date,time

In [8]:
def news_articles():
  for year in range(2010,2010):
    for month in range(10,10):
      for day in range(1,1):
        information=retrieving_information_for_the_day(year,month,day)
        headlines=headlines(information)
        links,news_titles=links_and_headings(headlines)
        authors,headlines=authors_and_reprocessing_data(links)
        news_contents=retrieving_news_content(links,headlines)